# 9. Applying QC cut-offs and VEP annotations

This script reads in matrix tables, applies QC filters and adds VEP annotations so these tables are then ready for variants to be counted. 

For this stage I used a mem2_ssd1_v2_x8 instance with 10 nodes, and used this one instance to run over all chromosomes. The total cost for this stage was £13. QC'd matrix tables are saved out using DNAX. 

## Set up environment
Make sure you run this block only once. You'll get errors if you try to initialise Hail multiple times. If you do do this, you'll need to restart the kernel, and then initialise Hail only once. 


In [ ]:
# Initialise hail and spark logs? Running this cell will output a red-colored message- this is expected.
# The 'Welcome to Hail' message in the output will indicate that Hail is ready to use in the notebook.
import pyspark.sql

config = pyspark.SparkConf().setAll([('spark.kryoserializer.buffer.max', '128')])
sc = pyspark.SparkContext(conf=config) 

from pyspark.sql import SparkSession

import hail as hl
builder = (
    SparkSession
    .builder
    .enableHiveSupport()
)
spark = builder.getOrCreate()
hl.init(sc=sc)

import dxpy

## Read in files needed for annotations across all chromosomes 
Here, read in (and key correctly for annotation): 
- the IDs of unrelated samples you want to keep (from stage 6). 
- REVEL scores 
- GnomAD annotations

In [ ]:
# UNRELATED IDS
### Read in unrealted sample IDs to keep in (this file was created in stage 06).
samples_to_keep=hl.import_table(f"file:///mnt/project/WES_QC/3rd_degree_rel_ids_to_keep_500k.csv", delimiter=',')
samples_to_keep.count()
samples_to_keep=samples_to_keep.rename({'"x"':"s"})
samples_to_keep=samples_to_keep.key_by(samples_to_keep.s)

# REVEL SCORES 
# Read in REVEL file, defining the locus field as type = locus so it can be annotated to other loci
revel=hl.import_table(f"file:///mnt/project/matching_200k_analysis/revel_scores.tsv", 
                      delimiter= '\t',
                      types={'locus': hl.expr.types.tlocus(reference_genome='GRCh38'), 'alt': hl.tstr, 'REVEL_score': hl.tfloat64}) 
print(revel.count())
revel=revel.key_by(revel.locus, revel.alt)

# GNOMAD
# Read in gnomad file 
## In terminal read in file from project using 'dx download matching_200k_analysis/constraint_scores.tsv' 
constraint=hl.import_table(f"file:///mnt/project/matching_200k_analysis/constraint_scores.tsv", 
                           delimiter='\t',
                           types={'pLI': hl.tfloat64, 'oe_lof':hl.tfloat64, 'oe_lof_upper':hl.tfloat64})
constraint.count()
constraint=constraint.key_by('gene_id') # This is the ensembl id, so like ENSG000..... 
# Rename to match matrix table 
constraint=constraint.rename({"gene_id":"gene_id_worstCsq"})

## Read in matrix tables per chromosome, filter, and annotate 
This whole process is within a loop that happens per chromosome. Note that chromosome 1 is treated differently. Chromosome 1 was too big to effectively work with as one matrix table, so here it is split into two halves and then each half is worked with individually from this point. 


Each section of the loop is described below: 
- Reading in matrix table: Here, you read in the matrix table which was saved out in stage 5 from a DNAX database. First, set the chromosome for which you are running this script. When working with chromosome 1, the mt is split into 2 halves at this point.  

- Filter to unrelated samples: This section uses the IDs of the unrelated samples you read in above to filter the matrix table so it only contains unrealted samples.

- Add VEP annotations: This sections uses the VEP annotations saved out in stage 4 to annotate the matrix table with variant consequences. The key of these tables matches so this is a simple annotation to add. Then set up PTV, NS, miss and synonymous annotations and worst consequences, and save out the matrix table to reduce computational load for next steps. 

- Add REVEL annotations: This section uses the REVEL annotations you read in above to annotate the matrix table with REVEL scores. This table is keyed by locus and alternate allele, so we need to form an alternate allele column within our matrix table, and then key by it. Once the keys are the same, the annotation can be added.

- Add per gene constraint metrics: This section uses the GnomAD annotations you read in above to annotate the matrix table with LOEUF and pLI scores per gene. This requires keying by gene ID. Here, we also rekey the table so it is keyed as it originally was and then write out the matrix table and read it back in to speed up processing. 

- Variant QC: filter MT to remove low quality variants highlighted in stage 5. 

- Allele counts: Run variant_qc command in hail to redo allele counts in this high quality sample and variant matrix table. These counts will later be used to filter to rare variants. 

- Write out: Write the annotated and QC'd matrix table out to DNAX. 


In [ ]:
### Filter and annotate MTs 
chromosomes=list(range(1,23))

for chr in chromosomes:
    if chr == 1:
        print(f"Processing chromosome {chr} ...")

        ## First half
        ####### READ IN MT #########
        mt=hl.read_matrix_table(f"dnax://database-Ggy1X3QJ637qBKGypjy9y9f4/chromosome_{chr}_post_geno_sample_and_var_qc.mt")
        # Check this table is as you'd expect
        print(f"Size when initially read in: {mt.count()}")
        # Split into two halves 
        total_rows=mt.count_rows()
        split_index=total_rows//2
        print(split_index)
        mt=mt.add_row_index()
        # First half
        mt_first_half=mt.filter_rows(hl.int(mt.row_idx)<split_index)
        print(mt_first_half.count())
        # Check you're happy with this split (that it follows on from each bit correctly)
        mt_first_half.tail(10)
        mt_first_half.checkpoint(f'chr{chr}_1sthalf.mt', overwrite=True)
        mt=hl.read_matrix_table(f'chr{chr}_1sthalf.mt')

        
        ####### REMOVE RELATED SAMPLES ######
        mt = mt.filter_cols((hl.is_defined(samples_to_keep[mt.s])))
        print(f'MT size after removing related samples: {mt.count()}')


        ####### ADD VEP ANNOTATIONS #########
        # Read in VEP annotations file 
        annotations = hl.read_table(f"dnax://database-Ggy5qPjJ637zF5j5yyX9bGxG/chr_{chr}_annotations_LoFTEE_updated.ht")
        print(f'Total annotations file size (should be bigger than mt): {annotations.count()}') # This should be bigger than n variants you have, as its from an unQCd version of this file
        # Set up annotations
        PTV_annotations = hl.set(["splice_acceptor_variant", "splice_donor_variant", "stop_gained", "frameshift_variant"])
        NS_annotations = hl.set(["splice_acceptor_variant", "splice_donor_variant", "stop_gained", "inframe_insertion", "inframe_deletion", "inframe_insertion", "missense_variant", "stop_lost", "start_lost", "frameshift_variant"])
        Miss_annotations = hl.set(["missense_variant"])
        S_annotations = hl.set(["synonymous_variant"])
        # Annotate
        annotated_mt = annotated_mt.annotate_rows(LoF_worstCsq = (PTV_annotations.contains(annotated_mt.vep.most_severe_consequence)),
                          NS_worstCsq = (NS_annotations.contains(annotated_mt.vep.most_severe_consequence)),
                          Miss_worstCsq = (Miss_annotations.contains(annotated_mt.vep.most_severe_consequence)),
                          Syn_worstCsq = (S_annotations.contains(annotated_mt.vep.most_severe_consequence)),
                          gene_symbol_worstCsq = (annotated_mt.vep.transcript_consequences.find(lambda x : x.consequence_terms.contains(annotated_mt.vep.most_severe_consequence)).gene_symbol),
                          gene_id_worstCsq = (annotated_mt.vep.transcript_consequences.find(lambda x : x.consequence_terms.contains(annotated_mt.vep.most_severe_consequence)).gene_id)
                         )
        print(f'Structure of VEP annotated MT (check this has all annotations you want!): {annotated_mt.describe()}') # Check these annotations have all been added


        ######## ADD REVEL ANNOTATIONS #######
        s=annotated_mt.alleles[1]
        annotated_mt=annotated_mt.annotate_rows(alt=s)
        # Rekey annotated mt
        annotated_mt=annotated_mt.key_rows_by(annotated_mt.locus, annotated_mt.alt)
        # Annotate
        vep_revel_mt=annotated_mt.annotate_rows(REVEL_score=revel[annotated_mt.locus, annotated_mt.alt].REVEL_score)
        print(f'Structure of REVEL annotated MT (check this has all annotations you want!): {vep_revel_mt.describe()}') # Check these annotations have all been added


        ####### ADD GNOMAD ANNOTATIONS ########
        #Rekey to match 
        vep_revel_mt=vep_revel_mt.key_rows_by(vep_revel_mt.gene_id_worstCsq)
        # Annotate 
        vep_revel_constraint_mt=vep_revel_mt.annotate_rows(LoF_gene_pLI_score =constraint[vep_revel_mt.gene_id_worstCsq].pLI)
        vep_revel_constraint_mt=vep_revel_constraint_mt.annotate_rows(LoF_gene_pLI_LoFi =constraint[vep_revel_constraint_mt.gene_id_worstCsq].pli_9)
        vep_revel_constraint_mt=vep_revel_constraint_mt.annotate_rows(LoF_gene_oe_lof_upper =constraint[vep_revel_constraint_mt.gene_id_worstCsq].oe_lof_upper)
        vep_revel_constraint_mt=vep_revel_constraint_mt.annotate_rows(LoF_gene_LOEUF_LoFi =constraint[vep_revel_constraint_mt.gene_id_worstCsq].loeuf_35)
        print(f'Structure of constraint annotated MT (check this has all annotations you want!): {vep_revel_constraint_mt.describe()}') # Check these annotations have all been added
        # Checkpoint here to speed up processing
        vep_revel_constraint_mt.checkpoint(f'vep_annotated_mt_chr{chr}_1sthalf.mt', overwrite=True)
        mt=hl.read_matrix_table(f'vep_annotated_mt_chr{chr}_1sthalf.mt')


        ####### VARIANT QC #########
        # Remove variants based on final QC stages
        print(f'n variants in chromosome {chr} first half:{mt.count_rows()}') 
        mt=mt.filter_rows(mt.variant_gq_over_40==True)
        print(f'n variants in chromosome {chr} first half with variant GQ > 40: {mt.count_rows()}')
        mt=mt.filter_rows(mt.variant_call_rate_over_90_percent==True)
        print(f'n variants in chromosome {chr} first half with call rate > 0.9: {mt.count_rows()}')


        ####### ALLELE COUNTS #####
        # Rerun variant QC command so allele counts you work with are post all QC 
        mt = hl.variant_qc(mt)
        print(f'Variant QC run and allele counts updated for chr {chr} first half. MT sized {mt.count()}')
    

        ####### SAVE OUT MT ########
        #Write out matrix table here so it can be read in from here for future counts!
        db_name = f"mt_count_ready"
        mt_name = f"chr_{chr}_first_half_ready_for_counts.mt"
        # Create database in DNAX
        stmt = f"CREATE DATABASE IF NOT EXISTS {db_name} LOCATION 'dnax://'"
        print(stmt)
        spark.sql(stmt).show()
        # Store MT in DNAX
        import dxpy
        # Find database ID of newly created database using dxpy method
        db_uri = dxpy.find_one_data_object(name=f"{db_name}", classname="database")['id']
        url = f"dnax://{db_uri}/{mt_name}"
        mt.checkpoint(url) 






        ## Second half
        ######### READ IN MT ########## 
        mt=hl.read_matrix_table(f"dnax://database-Ggy1X3QJ637qBKGypjy9y9f4/chromosome_{chr}_post_geno_sample_and_var_qc.mt")
        # Check this table is as you'd expect
        print(f"Size when initially read in: {mt.count()}")
        # Split into two halves 
        total_rows=mt.count_rows()
        split_index=total_rows//2
        print(split_index)
        mt=mt.add_row_index()
        # Second half
        mt_second_half=mt.filter_rows(hl.int(mt.row_idx)>=split_index) 
        print(mt_second_half.count())
        # Check you're happy with this split (that it follows on from each bit correctly)
        mt_second_half.head(10)
        mt_first_half.checkpoint(f'chr{chr}_2ndhalf.mt', overwrite=True)
        mt=hl.read_matrix_table(f'chr{chr}_2ndhalf.mt')

        
        ####### REMOVE RELATED SAMPLES ######
        mt = mt.filter_cols((hl.is_defined(samples_to_keep[mt.s])))
        print(f'MT size after removing related samples: {mt.count()}')


        ####### ADD VEP ANNOTATIONS #########
        # Read in VEP annotations file 
        annotations = hl.read_table(f"dnax://database-Ggy5qPjJ637zF5j5yyX9bGxG/chr_{chr}_annotations_LoFTEE_updated.ht")
        print(f'Total annotations file size (should be bigger than mt): {annotations.count()}') # This should be bigger than n variants you have, as its from an unQCd version of this file
        # Set up annotations
        PTV_annotations = hl.set(["splice_acceptor_variant", "splice_donor_variant", "stop_gained", "frameshift_variant"])
        NS_annotations = hl.set(["splice_acceptor_variant", "splice_donor_variant", "stop_gained", "inframe_insertion", "inframe_deletion", "inframe_insertion", "missense_variant", "stop_lost", "start_lost", "frameshift_variant"])
        Miss_annotations = hl.set(["missense_variant"])
        S_annotations = hl.set(["synonymous_variant"])
        # Annotate
        annotated_mt = annotated_mt.annotate_rows(LoF_worstCsq = (PTV_annotations.contains(annotated_mt.vep.most_severe_consequence)),
                          NS_worstCsq = (NS_annotations.contains(annotated_mt.vep.most_severe_consequence)),
                          Miss_worstCsq = (Miss_annotations.contains(annotated_mt.vep.most_severe_consequence)),
                          Syn_worstCsq = (S_annotations.contains(annotated_mt.vep.most_severe_consequence)),
                          gene_symbol_worstCsq = (annotated_mt.vep.transcript_consequences.find(lambda x : x.consequence_terms.contains(annotated_mt.vep.most_severe_consequence)).gene_symbol),
                          gene_id_worstCsq = (annotated_mt.vep.transcript_consequences.find(lambda x : x.consequence_terms.contains(annotated_mt.vep.most_severe_consequence)).gene_id)
                         )
        print(f'Structure of VEP annotated MT (check this has all annotations you want!): {annotated_mt.describe()}') # Check these annotations have all been added


        ######## ADD REVEL ANNOTATIONS #######
        s=annotated_mt.alleles[1]
        annotated_mt=annotated_mt.annotate_rows(alt=s)
        # Rekey annotated mt
        annotated_mt=annotated_mt.key_rows_by(annotated_mt.locus, annotated_mt.alt)
        # Annotate
        vep_revel_mt=annotated_mt.annotate_rows(REVEL_score=revel[annotated_mt.locus, annotated_mt.alt].REVEL_score)
        print(f'Structure of REVEL annotated MT (check this has all annotations you want!): {vep_revel_mt.describe()}') # Check these annotations have all been added


        ####### ADD GNOMAD ANNOTATIONS ########
        #Rekey to match 
        vep_revel_mt=vep_revel_mt.key_rows_by(vep_revel_mt.gene_id_worstCsq)
        # Annotate 
        vep_revel_constraint_mt=vep_revel_mt.annotate_rows(LoF_gene_pLI_score =constraint[vep_revel_mt.gene_id_worstCsq].pLI)
        vep_revel_constraint_mt=vep_revel_constraint_mt.annotate_rows(LoF_gene_pLI_LoFi =constraint[vep_revel_constraint_mt.gene_id_worstCsq].pli_9)
        vep_revel_constraint_mt=vep_revel_constraint_mt.annotate_rows(LoF_gene_oe_lof_upper =constraint[vep_revel_constraint_mt.gene_id_worstCsq].oe_lof_upper)
        vep_revel_constraint_mt=vep_revel_constraint_mt.annotate_rows(LoF_gene_LOEUF_LoFi =constraint[vep_revel_constraint_mt.gene_id_worstCsq].loeuf_35)
        print(f'Structure of constraint annotated MT (check this has all annotations you want!): {vep_revel_constraint_mt.describe()}') # Check these annotations have all been added
        # Checkpoint here to speed up processing
        vep_revel_constraint_mt.checkpoint(f'vep_annotated_mt_chr{chr}_2ndhalf.mt', overwrite=True)
        mt=hl.read_matrix_table(f'vep_annotated_mt_chr{chr}_2ndhalf.mt')


        ####### VARIANT QC #########
        # Remove variants based on final QC stages
        print(f'n variants in chromosome {chr} second half:{mt.count_rows()}') 
        mt=mt.filter_rows(mt.variant_gq_over_40==True)
        print(f'n variants in chromosome {chr} second half with variant GQ > 40: {mt.count_rows()}')
        mt=mt.filter_rows(mt.variant_call_rate_over_90_percent==True)
        print(f'n variants in chromosome {chr} second half with call rate > 0.9: {mt.count_rows()}')


        ####### ALLELE COUNTS #####
        # Rerun variant QC command so allele counts you work with are post all QC 
        mt = hl.variant_qc(mt)
        print(f'Variant QC run and allele counts updated for chr {chr} second half. MT sized {mt.count()}')
    

        ####### SAVE OUT MT ########
        #Write out matrix table here so it can be read in from here for future counts!
        db_name = f"mt_count_ready"
        mt_name = f"chr_{chr}_second_half_ready_for_counts.mt"
        # Create database in DNAX
        stmt = f"CREATE DATABASE IF NOT EXISTS {db_name} LOCATION 'dnax://'"
        print(stmt)
        spark.sql(stmt).show()
        # Store MT in DNAX
        import dxpy
        # Find database ID of newly created database using dxpy method
        db_uri = dxpy.find_one_data_object(name=f"{db_name}", classname="database")['id']
        url = f"dnax://{db_uri}/{mt_name}"
        mt.checkpoint(url) 



        print(f'Chromosome {chr} annotated and matrix tables (per half) written out!')
    else:
        print(f"Processing chromosome {chr} ...")

        ####### READ IN MT #########
        mt=hl.read_matrix_table(f"dnax://database-Ggy1X3QJ637qBKGypjy9y9f4/chromosome_{chr}_post_geno_sample_and_var_qc.mt")
        # Check this table is as you'd expect
        print(f"Size when initially read in: {mt.count()}")

        ####### REMOVE RELATED SAMPLES ######
        mt = mt.filter_cols((hl.is_defined(samples_to_keep[mt.s])))
        print(f'MT size after removing related samples: {mt.count()}')


        ####### ADD VEP ANNOTATIONS #########
        # Read in VEP annotations file 
        annotations = hl.read_table(f"dnax://database-Ggy5qPjJ637zF5j5yyX9bGxG/chr_{chr}_annotations_LoFTEE_updated.ht")
        print(f'Total annotations file size (should be bigger than mt): {annotations.count()}') # This should be bigger than n variants you have, as its from an unQCd version of this file
        # Set up annotations
        PTV_annotations = hl.set(["splice_acceptor_variant", "splice_donor_variant", "stop_gained", "frameshift_variant"])
        NS_annotations = hl.set(["splice_acceptor_variant", "splice_donor_variant", "stop_gained", "inframe_insertion", "inframe_deletion", "inframe_insertion", "missense_variant", "stop_lost", "start_lost", "frameshift_variant"])
        Miss_annotations = hl.set(["missense_variant"])
        S_annotations = hl.set(["synonymous_variant"])
        # Annotate
        annotated_mt = annotated_mt.annotate_rows(LoF_worstCsq = (PTV_annotations.contains(annotated_mt.vep.most_severe_consequence)),
                          NS_worstCsq = (NS_annotations.contains(annotated_mt.vep.most_severe_consequence)),
                          Miss_worstCsq = (Miss_annotations.contains(annotated_mt.vep.most_severe_consequence)),
                          Syn_worstCsq = (S_annotations.contains(annotated_mt.vep.most_severe_consequence)),
                          gene_symbol_worstCsq = (annotated_mt.vep.transcript_consequences.find(lambda x : x.consequence_terms.contains(annotated_mt.vep.most_severe_consequence)).gene_symbol),
                          gene_id_worstCsq = (annotated_mt.vep.transcript_consequences.find(lambda x : x.consequence_terms.contains(annotated_mt.vep.most_severe_consequence)).gene_id)
                         )
        print(f'Structure of VEP annotated MT (check this has all annotations you want!): {annotated_mt.describe()}') # Check these annotations have all been added


        ######## ADD REVEL ANNOTATIONS #######
        s=annotated_mt.alleles[1]
        annotated_mt=annotated_mt.annotate_rows(alt=s)
        # Rekey annotated mt
        annotated_mt=annotated_mt.key_rows_by(annotated_mt.locus, annotated_mt.alt)
        # Annotate
        vep_revel_mt=annotated_mt.annotate_rows(REVEL_score=revel[annotated_mt.locus, annotated_mt.alt].REVEL_score)
        print(f'Structure of REVEL annotated MT (check this has all annotations you want!): {vep_revel_mt.describe()}') # Check these annotations have all been added


        ####### ADD GNOMAD ANNOTATIONS ########
        #Rekey to match 
        vep_revel_mt=vep_revel_mt.key_rows_by(vep_revel_mt.gene_id_worstCsq)
        # Annotate 
        vep_revel_constraint_mt=vep_revel_mt.annotate_rows(LoF_gene_pLI_score =constraint[vep_revel_mt.gene_id_worstCsq].pLI)
        vep_revel_constraint_mt=vep_revel_constraint_mt.annotate_rows(LoF_gene_pLI_LoFi =constraint[vep_revel_constraint_mt.gene_id_worstCsq].pli_9)
        vep_revel_constraint_mt=vep_revel_constraint_mt.annotate_rows(LoF_gene_oe_lof_upper =constraint[vep_revel_constraint_mt.gene_id_worstCsq].oe_lof_upper)
        vep_revel_constraint_mt=vep_revel_constraint_mt.annotate_rows(LoF_gene_LOEUF_LoFi =constraint[vep_revel_constraint_mt.gene_id_worstCsq].loeuf_35)
        print(f'Structure of constraint annotated MT (check this has all annotations you want!): {vep_revel_constraint_mt.describe()}') # Check these annotations have all been added
        # Checkpoint here to speed up processing
        vep_revel_constraint_mt.checkpoint(f'vep_annotated_mt_chr{chr}.mt', overwrite=True)
        mt=hl.read_matrix_table(f'vep_annotated_mt_chr{chr}.mt')


        ####### VARIANT QC #########
        # Remove variants based on final QC stages
        print(f'n variants in chromosome {chr}:{mt.count_rows()}') 
        mt=mt.filter_rows(mt.variant_gq_over_40==True)
        print(f'n variants in chromosome {chr} with variant GQ > 40: {mt.count_rows()}')
        mt=mt.filter_rows(mt.variant_call_rate_over_90_percent==True)
        print(f'n variants in chromosome {chr} with call rate > 0.9: {mt.count_rows()}')


        ####### ALLELE COUNTS #####
        # Rerun variant QC command so allele counts you work with are post all QC 
        mt = hl.variant_qc(mt)
        print(f'Variant QC run and allele counts updated for chr {chr}. MT sized {mt.count()}')
    

        ####### SAVE OUT MT ########
        #Write out matrix table here so it can be read in from here for future counts!
        db_name = f"mt_count_ready"
        mt_name = f"chr_{chr}_ready_for_counts.mt"
        # Create database in DNAX
        stmt = f"CREATE DATABASE IF NOT EXISTS {db_name} LOCATION 'dnax://'"
        print(stmt)
        spark.sql(stmt).show()
        # Store MT in DNAX
        import dxpy
        # Find database ID of newly created database using dxpy method
        db_uri = dxpy.find_one_data_object(name=f"{db_name}", classname="database")['id']
        url = f"dnax://{db_uri}/{mt_name}"
        mt.checkpoint(url) 


        print(f'Chromosome {chr} annotated and matrix table written out!')